# LlamaIndex: AgentWorkflow Basic Introduction

- AgentWorkflow Basic Introduction<br>
  https://docs.llamaindex.ai/en/stable/examples/agent/agent_workflow_basic/
- **LlamaHub Integrations**<br>
  https://llamahub.ai/
- Examples - LLMs (**OpenAI**)<br>
  https://docs.llamaindex.ai/en/stable/examples/llm/openai/

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## SETUP (LlamaIndex)

In [2]:
# %pip install llama-index
# %pip install llama-index-llms-openai
# %pip install tavily-python

In [3]:
from llama_index.llms.openai import OpenAI

# llm = OpenAI(model="gpt-4o-mini", api_key="sk-...")
llm = OpenAI(model="gpt-4o-mini")

In [4]:
# Creating a tool
import os
from tavily import AsyncTavilyClient

tavily_api_key = os.getenv("TAVILY_API_KEY")

async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=tavily_api_key)
    return str(await client.search(query))

In [6]:
from llama_index.core.agent.workflow import AgentWorkflow

workflow = AgentWorkflow.from_tools_or_functions(
    [search_web],
    llm=llm,
    system_prompt="You are a helpful assistant that can search the web for information.",
)

## Running the Agent

In [7]:
# response = await workflow.run(user_msg="What is the weather in San Francisco?")
response = await workflow.run(user_msg="What is the weather in Prague?")
print(str(response))

The current weather in San Francisco is as follows:

- **Temperature**: 13.9°C (57°F)
- **Condition**: Partly cloudy
- **Wind**: 10.5 mph (16.9 kph) from the SSW
- **Humidity**: 87%
- **Visibility**: 16 km (9 miles)
- **Feels Like**: 12.6°C (54.6°F)

For more details, you can check the full report [here](https://www.weatherapi.com/).


## Maintaining State

In [8]:
from llama_index.core.workflow import Context

ctx = Context(workflow)

response = await workflow.run(
    user_msg="My name is Logan, nice to meet you!", ctx=ctx
)
print(str(response))

Nice to meet you, Logan! How can I assist you today?


In [9]:
response = await workflow.run(user_msg="What is my name?", ctx=ctx)
print(str(response))

Your name is Logan.


In [10]:
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer

ctx_dict = ctx.to_dict(serializer=JsonSerializer())

restored_ctx = Context.from_dict(
    workflow, ctx_dict, serializer=JsonSerializer()
)

In [16]:
print(type(ctx_dict))
print(len(ctx_dict))
ctx_dict.keys()
# ctx_dict

<class 'dict'>
10


dict_keys(['globals', 'streaming_queue', 'queues', 'stepwise', 'events_buffer', 'in_progress', 'accepted_events', 'broker_log', 'waiter_id', 'is_running'])

In [17]:
response = await workflow.run(
    user_msg="Do you still remember my name?", ctx=restored_ctx
)
print(str(response))

Yes, I remember your name is Logan.


## Streaming

In [18]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = workflow.run(user_msg="What is the weather in Saskatoon?")

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)
        # print(event.response)  # the current full response
        # print(event.raw)  # the raw llm api response
        # print(event.current_agent_name)  # the current agent name
    # elif isinstance(event, AgentInput):
    #    print(event.input)  # the current input messages
    #    print(event.current_agent_name)  # the current agent name
    # elif isinstance(event, AgentOutput):
    #    print(event.response)  # the current full response
    #    print(event.tool_calls)  # the selected tool calls, if any
    #    print(event.raw)  # the raw llm api response
    # elif isinstance(event, ToolCallResult):
    #    print(event.tool_name)  # the tool name
    #    print(event.tool_kwargs)  # the tool kwargs
    #    print(event.tool_output)  # the tool output
    # elif isinstance(event, ToolCall):
    #     print(event.tool_name)  # the tool name
    #     print(event.tool_kwargs)  # the tool kwargs

The current weather in Saskatoon is as follows:

- **Temperature**: -1.8°C (28.8°F)
- **Condition**: Overcast
- **Wind**: 9.8 mph (15.8 kph) from the ESE
- **Humidity**: 69%
- **Feels Like**: -6.8°C (19.8°F)
- **Visibility**: 24 km

For more details, you can check the full weather report [here](https://www.weatherapi.com/).

## Tools and State

In [19]:
from llama_index.core.workflow import Context

async def set_name(ctx: Context, name: str) -> str:
    state = await ctx.get("state")
    state["name"] = name
    await ctx.set("state", state)
    return f"Name set to {name}"

workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

ctx = Context(workflow)

In [20]:
response = await workflow.run(user_msg="My name is Logan", ctx=ctx)
print(str(response))

state = await ctx.get("state")
print(state["name"])

Your name has been set to Logan.
Logan


## Human in the Loop

In [21]:
from llama_index.core.workflow import (
    Context,
    InputRequiredEvent,
    HumanResponseEvent,
)

async def dangerous_task(ctx: Context) -> str:
    """A dangerous task that requires human confirmation."""
    ctx.write_event_to_stream(
        InputRequiredEvent(
            prefix="Are you sure you want to proceed?",
            user_name="Logan",
        )
    )

    response = await ctx.wait_for_event(
        HumanResponseEvent, requirements={"user_name": "Logan"}
    )
    if response.response == "yes":
        return "Dangerous task completed successfully."
    else:
        return "Dangerous task aborted."

workflow = AgentWorkflow.from_tools_or_functions(
    [dangerous_task],
    llm=llm,
    system_prompt="You are a helpful assistant that can perform dangerous tasks.",
)

In [22]:
handler = workflow.run(user_msg="I want to proceed with the dangerous task.")

async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        response = input(event.prefix).strip().lower()
        handler.ctx.send_event(
            HumanResponseEvent(
                response=response,
                user_name=event.user_name,
            )
        )

# response = await handler
# print(str(response))

Are you sure you want to proceed? yes


In [23]:
response = await handler
print(str(response))

The dangerous task has been completed successfully. If you need anything else, feel free to ask!


In [24]:
from llama_index.core.workflow import JsonSerializer

handler = workflow.run(user_msg="I want to proceed with the dangerous task.")

input_ev = None
async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        input_ev = event
        break

# save the context somewhere for later
ctx_dict = handler.ctx.to_dict(serializer=JsonSerializer())

# get the response from the user
response_str = input(input_ev.prefix).strip().lower()

# restore the workflow
restored_ctx = Context.from_dict(
    workflow, ctx_dict, serializer=JsonSerializer()
)

handler = workflow.run(ctx=restored_ctx)
handler.ctx.send_event(
    HumanResponseEvent(
        response=response_str,
        user_name=input_ev.user_name,
    )
)

# response = await handler
# print(str(response))

Are you sure you want to proceed? yes


In [25]:
response = await handler
print(str(response))

The dangerous task has been completed successfully. If you need anything else, feel free to ask!
